# 2018-06-08 - Comparaison modèle/humain
# Pour génerer les MC  : 2018-06-17 Notebook

# Le convo

In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets

data_transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.Resize((128,128)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5,0.5), (0.5,0.5,0.5))])

#train
train_set = datasets.ImageFolder(root='16_clouds_easy',
                                transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                             batch_size=4, shuffle=True,
                                             num_workers=1, drop_last = True)

#test
test_set = datasets.ImageFolder(root='16_clouds_easy_test',
                                transform=data_transform)
test_loader = torch.utils.data.DataLoader(test_set,
                                             batch_size=4,shuffle=True,
                                             num_workers=1, drop_last = True)

In [2]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 20)
        self.pool = nn.MaxPool2d(2,2)

        self.fc3 = nn.Linear(17496,1000)
        self.dropout = nn.Dropout(0.2)

        self.outlayer = nn.Linear(1000,16)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))

        x = x.view(x.size(0), -1) #reshape from conv to linear

        x = F.leaky_relu(self.fc3(x))
        x = self.dropout(x)

        x = self.outlayer(x)
        return x
        
model = Net()
print(model)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(20, 20), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc3): Linear(in_features=17496, out_features=1000, bias=True)
  (dropout): Dropout(p=0.2)
  (outlayer): Linear(in_features=1000, out_features=16, bias=True)
)


In [3]:
#freeze layers after and including freezing_layer+1 (layers start at 0)
def freeze_layers(freezing_layer, nn_model = model) :
    for count,child in enumerate(model.children()) :
        if  count < freezing_layer+1 : #to freeze at iteration 1
            print("Layer no. %s -- %s -- NOT FROZEN"% (count,child))
            for param in child.parameters():
                param.requires_grad = True
        else :
            print("Layer no. %s -- %s -- FROZEN"%(count, child))
            for param in child.parameters():
                param.requires_grad = False

#freeze all the layers except the unfrozen one
def freeze_all_layers(unfrozen_layer, nn_model = model) :
    for count,child in enumerate(model.children()) :
        if count == unfrozen_layer :
            print("Layer no. %s -- %s -- NOT FROZEN"% (count,child))
            for param in child.parameters():
                param.requires_grad = True
        else :
            print("Layer no. %s -- %s -- FROZEN"%(count, child))
            for param in child.parameters():
                param.requires_grad = False

#unfreeze every layers
def layers_microwave(nn_model = model) :
    for count,child in enumerate(model.children()) :
        for param in child.parameters():
            param.requires_grad = True

In [ ]:
import torch.optim as optim
import time

#unfreeze before starting
layers_microwave(model)

criterion = nn.CrossEntropyLoss() #loss criterion
optimizer = optim.SGD(params = model.parameters(),lr=0.001, momentum=0.9)
epochs = 2 #nbr of epochs per layer
model_size = 4 #nbr of layers

print_interval = 50 #prints every p_i*4
tempo = []
acc = []

start_time = time.time()
print("Started training")

for epoch in range(epochs):  # nbr epochs
    for batch_idx, (data, target) in enumerate(train_loader): #nbr batch,in,out
        data, target = Variable(data), Variable(target)
        #On resize pour la sortie


        #init l'entrainement
        optimizer.zero_grad()
        net_out = model(data)

        loss = criterion(net_out, target)
        loss.backward()
        optimizer.step()

        #afficher la progression
        if batch_idx % print_interval == 0:
            #le print statement le plus illisible du monde
            print('Epoch: {} [{}/{} ({:.0f}%)]\t\tLoss: {:.6f}'.format(
                    epoch+1, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0]))
    tempo.append(epoch)
    acc.append(loss.data[0])


print("Finished training in  %.3f seconds " % (time.time() - start_time))

In [ ]:
# delete
test_loss = 0
correct = 0
model = model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)

    net_out = model(data)
    
    #somme des pertes du batch
    test_loss += criterion(net_out, target).data[0]
    pred = net_out.data.max(1)[1] #prediction
    correct += pred.eq(target.data).sum() #output du réseau

test_loss /= len(test_loader.dataset) #loss = loss/length set
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

## Convo - Psychophysique

In [ ]:
#j'ai augmenté les B_theta et la longueur du test pour lisser les courbes
exp_info = 'convo_model_HIRES'
print (exp_info)

On défini la génération de MotionClouds :

In [ ]:
import numpy as np
import MotionClouds as mc
import matplotlib.pyplot as plt
import os
import imageio
import random

downscale = 1
fig_width = 21
fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)

# generates a cloud of given theta and b_theta
def generate_random_cloud(theta, B_theta):
    mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0.,
                             B_V=0, theta=theta, B_theta=B_theta)
    im = mc.random_cloud(mc_i)
    im = (mc.rectif(im) * 255).astype('uint8')
    fname = './tmp/%s_%s.png' % (theta, B_theta)
    imageio.imwrite(fname, im[:, :, 0])
    return fname

On définit les paramètres et on teste :

In [ ]:
from psychopy import visual, core, event
import MotionClouds as MC
import matplotlib.pyplot as plt
import matplotlib.image as mpimage
from PIL import Image

test_length = 600
MC1 = generate_random_cloud(np.pi/2, B_theta=np.pi/2)
model = model.eval()


ans_list = []
N_B_thetas = 15
B_thetas = np.pi*np.logspace(-6, -1, N_B_thetas, base=2)
    
std_theta = np.pi/6

for trial in range(test_length):
    theta = np.clip(std_theta *  np.random.randn(), -np.pi/4, np.pi/4)

    # MC generation
    B_theta = B_thetas[random.randint(0, N_B_thetas-1)]

    MC1 = generate_random_cloud(np.pi/2, B_theta=B_theta)
    MC2 = generate_random_cloud(np.pi/2 - theta, B_theta=B_theta)  # if shift = 2

    varimgmc2 = data_transform(Image.open(MC2))
    #varimgmc2 = data_transform(Image.open('16_clouds_easy/0.0/B0 1.40625.png'))
    varimgmc2 = Variable(varimgmc2)
    varimgmc2 = varimgmc2.unsqueeze(0)
    
    net_ans = model(varimgmc2)
    pred = net_ans.data.max(1)[1] #prediction
    #print(pred)
    #correct = (np.sign(theta) > 0) and (pred[0]>8)
    if np.sign(theta) > 0 and pred[0] > 8 :
        correct = True
    if np.sign(theta) < 0 and pred[0] <= 8:
        correct = True
    else :
        correct = False
    print('At trial ', trial, 'Angle=', '%3.3f' % (theta*180/np.pi), 'answer is ', pred[0], '(correct=', correct, '); bandwidth=', '%.3f' % (B_theta*180/np.pi))

    # Output shape per trial is : trial number, shift direction, answered shift and b_theta
    if pred[0] >8 :
        ans_list.append([trial, theta, 'left', B_theta])
    if pred[0] <= 8 :
        ans_list.append([trial, theta, 'right', B_theta])


import pickle
pickle.dump(ans_list, open('./psychophysics_data/Psy_discrim_final_%s.p' % exp_info, 'wb'))

# Le Ring

In [13]:
"""
Created on Tue May 15 11:25:41 2018

@author: hugo
"""
from torch.autograd import Variable
import torch 
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
import torch.nn.functional as F

#Transform
data_transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.Resize((128,128)),
    transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5,0.5), (0.5,0.5,0.5))])

#Train
train_set = datasets.ImageFolder(root='16_clouds_easy',
                                transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                             batch_size=8, shuffle=True,
                                             num_workers=1)

#Test
test_set = datasets.ImageFolder(root='16_clouds_easy_test',
                                transform=data_transform)
test_loader = torch.utils.data.DataLoader(test_set,
                                             batch_size=8,shuffle=False,
                                             num_workers=1)

###################################################################################

# Hyper-parameters
sequence_length = 49
input_size = 49
hidden_size = 128
num_layers = 2
num_classes = 16
batch_size = 8
num_epochs = 60
learning_rate = 0.003

###################################################################################

class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout = 0.3)
        self.conv1 = nn.Conv2d(1,8,30)
        self.conv2 = nn.Conv2d(1,8,30)
        self.pool1 = nn.MaxPool2d(2,2)
        self.pool2 = nn.MaxPool2d(2,2)
        
        #self.drop = nn.Dropout(0.2)

        self.fc = nn.Linear(113 , 16)  # 2 for bidirection
    
    def forward(self, x):
        #print('Input %s' % str(x.size()))

        out = self.pool1(F.relu(self.conv1(x)))
        #print('Convoluted %s' % str(x.size()))

        # Init
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)
        h0 = Variable(h0).cuda()
        c0 = Variable(c0).cuda()
        
        out = out[:,-1,:,:]
        #print('Resized for RNN %s' % str(x.size()))
        
        # LSTM forward
        out, _ = self.rnn(out, (h0,c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        #print('After RNN %s' % str(out.size()))
        
        out = out.unsqueeze(1)
        #print(out.size())
        out = self.pool2(F.relu(self.conv2(out)))
        #print(out.size())
        out = out[:,-1,:,:]
        #Dropout
        #out = self.drop(out)
        #print(out.size())
        
        out = out[:,-1,:]
        # LSTM output
        out = self.fc(out)
        #print('Reshaped for output %s \n'%  str(out.size()))

        return out

model = BiRNN(input_size, hidden_size, num_layers, num_classes).cuda()
print(model)

###################################################################################

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

###################################################################################

#plotting list
loss_list = []
time_list = []
t = 0

print("Start training")
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
     
    for i, (images, labels) in enumerate(train_loader):
        freeze_layers
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.data[0]))
            loss_list.append(loss.data[0])
            time_list.append(t)
            t+=1

correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images).cuda()
    labels = Variable(labels).cuda()

    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += predicted.eq(labels.data).sum()

print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

%matplotlib inline
import matplotlib.pyplot as plt
import np
plt.plot(np.asarray(time_list)[::4], np.asarray(loss_lis[::4]))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss for LSTM-CNN')
plt.show()

BiRNN(
  (rnn): LSTM(49, 128, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (conv1): Conv2d(1, 8, kernel_size=(30, 30), stride=(1, 1))
  (conv2): Conv2d(1, 8, kernel_size=(30, 30), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=113, out_features=16, bias=True)
)
Start training


/home/hugo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:134: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/hugo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:135: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch [1/60], Step [100/480], Loss: 2.0963
Epoch [1/60], Step [200/480], Loss: 2.4450
Epoch [1/60], Step [300/480], Loss: 2.2233


Process Process-62:
Traceback (most recent call last):
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
KeyboardInterrupt
  File "/home/hugo/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 110, in _worker_loop
    data_queue.put((idx, samples))
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/queues.py", line 341, in put
    obj = _ForkingPickler.dumps(obj)
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
  File "/home/hugo/anaconda3/lib/python3.6/copyreg.py", line 96, in _slotnames
    def _slotnames(cls):


KeyboardInterrupt: 

In [14]:
#NOW FOR THE PSYCHOPHYSICS
import numpy as np
import MotionClouds as mc
import matplotlib.pyplot as plt
import os
import imageio
import random

exp_info = 'model_ring_HIRES'
print (exp_info)

downscale = 1
fig_width = 21
fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)

# generates a cloud of given theta and b_theta
def generate_random_cloud(theta, B_theta):
    mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0.,
                             B_V=0, theta=theta, B_theta=B_theta)
    
    im = mc.random_cloud(mc_i)
    im = (mc.rectif(im) * 255).astype('uint8')
    fname = './tmp/%s_%s.bmp' % (theta, B_theta)
    imageio.imwrite(uri = fname,im = im[:, :, 0], format = 'bmp')
    return fname

model_ring_HIRES


In [10]:
from psychopy import visual, core, event
import MotionClouds as MC
import matplotlib.pyplot as plt
from PIL import Image
import os
import glob

files = glob.glob('./tmp/*')
if not os.path.exists('./tmp'): #si le folder n'existe pas on le crée
        os.makedirs('./tmp')
        
        
test_length = 600
MC1 = generate_random_cloud(np.pi/2, B_theta=np.pi/2)
model = model.eval()

ans_list = []
N_B_thetas = 15
B_thetas = np.pi*np.logspace(-6, -1, N_B_thetas, base=2)
    
std_theta = np.pi/6

for trial in range(test_length):
    theta = np.clip(std_theta *  np.random.randn(), -np.pi/4, np.pi/4)

    # MC generation
    B_theta = B_thetas[random.randint(0, N_B_thetas-1)]

    MC1 = generate_random_cloud(np.pi/2, B_theta=B_theta)
    MC2 = generate_random_cloud(np.pi/2 - theta, B_theta=B_theta)  # if shift = 2

    varimgmc2 = data_transform(Image.open(MC2))
    #varimgmc2 = data_transform(Image.open('16_clouds_easy/0.0/B0 1.40625.png'))
    varimgmc2 = Variable(varimgmc2).cuda()
    varimgmc2 = varimgmc2.unsqueeze(0)
    
    net_ans = model(varimgmc2)
    pred = net_ans.data.max(1)[1] #prediction
    #print(pred)
    #correct = (np.sign(theta) > 0) and (pred[0]>8)
    if np.sign(theta) > 0 and pred[0] > 8 :
        correct = True
    if np.sign(theta) < 0 and pred[0] <= 8:
        correct = True
    else :
        correct = False
    print('At trial ', trial, 'Angle=', '%3.3f' % (theta*180/np.pi), 'answer is ', pred[0], '(correct=', correct, '); bandwidth=', '%.3f' % (B_theta*180/np.pi))

    # Output shape per trial is : trial number, shift direction, answered shift and b_theta
    if pred[0] >8 :
        ans_list.append([trial, theta, 'left', B_theta])
    if pred[0] <= 8 :
        ans_list.append([trial, theta, 'right', B_theta])

#cleanup temp
for f in files:
    os.remove(f)

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
At trial  0 Angle= -9.710 answer is  tensor(2, device='cuda:0') (correct= True ); bandwidth= 90.000
At trial  1 Angle= -5.476 answer is  tensor(8, device='cuda:0') (correct= True ); bandwidth= 5.911
At trial  2 Angle= 42.652 answer is  tensor(4, device='cuda:0') (correct= False ); bandwidth= 12.421
At trial  3 Angle= 31.829 answer is  tensor(5, device='cuda:0') (correct= False ); bandwidth= 12.421
At trial  4 Angle= 26.048 answer is  tensor(9, device='cuda:0') (correct= False ); bandwidth= 70.264
At trial  5 Angle= -27.937 answer is  tensor(6, device='cuda:0') (correct= True ); bandwidth= 26.103
At trial  6 Angle= 45.000 answer is  tensor(5, device='cuda:0') (correct= False ); bandwidth= 90.000
At trial  7 Angle= 4.591 answer is  tensor(7, device='cuda:0') (correct= False ); bandwidth= 4.614
At trial  8 Angle= 6.503 answer is  tensor(5, device='cuda:0') (correct= False ); bandwidth= 70.264
At trial  9 

At trial  83 Angle= 17.647 answer is  tensor(5, device='cuda:0') (correct= False ); bandwidth= 90.000
At trial  84 Angle= -12.858 answer is  tensor(5, device='cuda:0') (correct= True ); bandwidth= 70.264
At trial  85 Angle= -27.597 answer is  tensor(10, device='cuda:0') (correct= False ); bandwidth= 20.379
At trial  86 Angle= -6.842 answer is  tensor(5, device='cuda:0') (correct= True ); bandwidth= 70.264
At trial  87 Angle= -31.973 answer is  tensor(10, device='cuda:0') (correct= False ); bandwidth= 15.910
At trial  88 Angle= 7.578 answer is  tensor(0, device='cuda:0') (correct= False ); bandwidth= 2.812
At trial  89 Angle= 15.144 answer is  tensor(11, device='cuda:0') (correct= False ); bandwidth= 4.614
At trial  90 Angle= 40.419 answer is  tensor(4, device='cuda:0') (correct= False ); bandwidth= 5.911
At trial  91 Angle= 1.306 answer is  tensor(9, device='cuda:0') (correct= False ); bandwidth= 42.826
At trial  92 Angle= 17.678 answer is  tensor(9, device='cuda:0') (correct= False );

At trial  169 Angle= -23.771 answer is  tensor(9, device='cuda:0') (correct= False ); bandwidth= 33.435
At trial  170 Angle= -31.599 answer is  tensor(10, device='cuda:0') (correct= False ); bandwidth= 3.602
At trial  171 Angle= -19.981 answer is  tensor(2, device='cuda:0') (correct= True ); bandwidth= 15.910
At trial  172 Angle= 27.428 answer is  tensor(5, device='cuda:0') (correct= False ); bandwidth= 90.000
At trial  173 Angle= 15.960 answer is  tensor(6, device='cuda:0') (correct= False ); bandwidth= 2.812
At trial  174 Angle= -45.000 answer is  tensor(5, device='cuda:0') (correct= True ); bandwidth= 54.856
At trial  175 Angle= 7.265 answer is  tensor(7, device='cuda:0') (correct= False ); bandwidth= 3.602
At trial  176 Angle= -20.745 answer is  tensor(9, device='cuda:0') (correct= False ); bandwidth= 5.911
At trial  177 Angle= -2.658 answer is  tensor(9, device='cuda:0') (correct= False ); bandwidth= 3.602
At trial  178 Angle= -11.256 answer is  tensor(3, device='cuda:0') (correct

At trial  256 Angle= 0.520 answer is  tensor(5, device='cuda:0') (correct= False ); bandwidth= 90.000
At trial  257 Angle= 11.677 answer is  tensor(11, device='cuda:0') (correct= False ); bandwidth= 9.697
At trial  258 Angle= -3.066 answer is  tensor(9, device='cuda:0') (correct= False ); bandwidth= 90.000
At trial  259 Angle= 6.995 answer is  tensor(7, device='cuda:0') (correct= False ); bandwidth= 5.911
At trial  260 Angle= -2.438 answer is  tensor(8, device='cuda:0') (correct= True ); bandwidth= 12.421
At trial  261 Angle= -27.655 answer is  tensor(6, device='cuda:0') (correct= True ); bandwidth= 33.435
At trial  262 Angle= 2.360 answer is  tensor(10, device='cuda:0') (correct= False ); bandwidth= 70.264
At trial  263 Angle= 40.952 answer is  tensor(5, device='cuda:0') (correct= False ); bandwidth= 3.602
At trial  264 Angle= -12.069 answer is  tensor(9, device='cuda:0') (correct= False ); bandwidth= 4.614
At trial  265 Angle= 21.083 answer is  tensor(8, device='cuda:0') (correct= Fa

At trial  339 Angle= 2.507 answer is  tensor(7, device='cuda:0') (correct= False ); bandwidth= 5.911
At trial  340 Angle= -16.593 answer is  tensor(7, device='cuda:0') (correct= True ); bandwidth= 15.910
At trial  341 Angle= 4.753 answer is  tensor(2, device='cuda:0') (correct= False ); bandwidth= 54.856
At trial  342 Angle= -19.721 answer is  tensor(9, device='cuda:0') (correct= False ); bandwidth= 4.614
At trial  343 Angle= -22.238 answer is  tensor(9, device='cuda:0') (correct= False ); bandwidth= 12.421
At trial  344 Angle= -26.665 answer is  tensor(6, device='cuda:0') (correct= True ); bandwidth= 42.826
At trial  345 Angle= -40.469 answer is  tensor(11, device='cuda:0') (correct= False ); bandwidth= 2.812
At trial  346 Angle= -45.000 answer is  tensor(11, device='cuda:0') (correct= False ); bandwidth= 33.435
At trial  347 Angle= -4.618 answer is  tensor(9, device='cuda:0') (correct= False ); bandwidth= 42.826
At trial  348 Angle= 10.569 answer is  tensor(2, device='cuda:0') (corre

At trial  422 Angle= 16.506 answer is  tensor(10, device='cuda:0') (correct= False ); bandwidth= 90.000
At trial  423 Angle= 9.588 answer is  tensor(9, device='cuda:0') (correct= False ); bandwidth= 20.379
At trial  424 Angle= 6.020 answer is  tensor(5, device='cuda:0') (correct= False ); bandwidth= 54.856
At trial  425 Angle= 45.000 answer is  tensor(4, device='cuda:0') (correct= False ); bandwidth= 12.421
At trial  426 Angle= 45.000 answer is  tensor(4, device='cuda:0') (correct= False ); bandwidth= 2.812
At trial  427 Angle= -43.691 answer is  tensor(9, device='cuda:0') (correct= False ); bandwidth= 70.264
At trial  428 Angle= -21.718 answer is  tensor(5, device='cuda:0') (correct= True ); bandwidth= 70.264
At trial  429 Angle= 13.348 answer is  tensor(11, device='cuda:0') (correct= False ); bandwidth= 4.614
At trial  430 Angle= 22.242 answer is  tensor(9, device='cuda:0') (correct= False ); bandwidth= 33.435
At trial  431 Angle= -18.994 answer is  tensor(2, device='cuda:0') (correc

At trial  508 Angle= 8.050 answer is  tensor(7, device='cuda:0') (correct= False ); bandwidth= 5.911
At trial  509 Angle= -27.107 answer is  tensor(10, device='cuda:0') (correct= False ); bandwidth= 5.911
At trial  510 Angle= 14.816 answer is  tensor(5, device='cuda:0') (correct= False ); bandwidth= 33.435
At trial  511 Angle= -7.289 answer is  tensor(9, device='cuda:0') (correct= False ); bandwidth= 20.379
At trial  512 Angle= 45.000 answer is  tensor(4, device='cuda:0') (correct= False ); bandwidth= 5.911
At trial  513 Angle= 8.700 answer is  tensor(7, device='cuda:0') (correct= False ); bandwidth= 15.910
At trial  514 Angle= -8.888 answer is  tensor(8, device='cuda:0') (correct= True ); bandwidth= 3.602
At trial  515 Angle= -14.467 answer is  tensor(8, device='cuda:0') (correct= True ); bandwidth= 5.911
At trial  516 Angle= 26.898 answer is  tensor(5, device='cuda:0') (correct= False ); bandwidth= 20.379
At trial  517 Angle= -32.972 answer is  tensor(9, device='cuda:0') (correct= Fa

At trial  593 Angle= -5.592 answer is  tensor(8, device='cuda:0') (correct= True ); bandwidth= 3.602
At trial  594 Angle= -45.000 answer is  tensor(0, device='cuda:0') (correct= True ); bandwidth= 4.614
At trial  595 Angle= -7.312 answer is  tensor(10, device='cuda:0') (correct= False ); bandwidth= 54.856
At trial  596 Angle= -21.946 answer is  tensor(10, device='cuda:0') (correct= False ); bandwidth= 33.435
At trial  597 Angle= 25.447 answer is  tensor(10, device='cuda:0') (correct= False ); bandwidth= 12.421
At trial  598 Angle= 45.000 answer is  tensor(4, device='cuda:0') (correct= False ); bandwidth= 2.812
At trial  599 Angle= -18.601 answer is  tensor(9, device='cuda:0') (correct= False ); bandwidth= 20.379


import pickle
pickle.dump(ans_list, open('./psychophysics_data/Psy_discrim_final_%s.p' % exp_info, 'wb'))

# Trash

In [ ]:
varimgmc2 = data_transform(Image.open('16_clouds_easy/2.722713633111154/B0 10.928413651643268.png'))
varimgmc2 = Variable(varimgmc2)
varimgmc2 = varimgmc2.unsqueeze(0)

net_ans = model(varimgmc2)
print(net_ans.data.max(1)[1])